<a href="https://colab.research.google.com/github/hosngholami/RecommenderSystem/blob/main/RecommenderSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [89]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install opendatasets
!pip install geopy
!pip install folium
!pip install -U pymoo
!pip install scikit-

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 12.5 MB/s eta 0:00:00
  Created wheel for grapheme: filename=grapheme-0.6.0-py3-none-any.whl size=210082 sha256=64d9914da0b57773c0c26bc6add491fdf5edf81b8764d34f12d6f52d6632835e
  Stored in directory: /root/.cache/pip/wheels/ee/3b/0b/1b865800e916d671a24028d884698674138632a83fdfad4926
Successfully built grapheme
ERROR: Invalid requirement: 'scikit-': Expected end or semicolon (after name and no valid version specifier)
    scikit-
          ^


In [4]:
import pandas as pd
import numpy as np
import random
from sklearn.neighbors import NearestNeighbors

pd.set_option("display.max_columns", 1000)
pd.set_option('display.max_rows', 1000)

In [5]:
hotel = pd.read_csv('/content/drive/MyDrive/Shiraz/hotel.csv')
historic =  pd.read_csv('/content/drive/MyDrive/Shiraz/historic.csv')
museum = pd.read_csv('/content/drive/MyDrive/Shiraz/museum.csv')
park = pd.read_csv('/content/drive/MyDrive/Shiraz/park.csv')
resturant = pd.read_csv('/content/drive/MyDrive/Shiraz/resturant.csv')

preference = pd.read_csv('/content/drive/MyDrive/Shiraz/preference.csv')
group = pd.read_csv('/content/drive/MyDrive/Shiraz/group.csv')
tag = pd.read_csv('/content/drive/MyDrive/Shiraz/tag.csv')

In [6]:
tag

,Unnamed: 0,Id,Title,Title:Fn
0,0,1,Historical,مکان‌های تاریخی (مانند قلعه‌ها، موزه‌ها، آثار ...
1,1,2,Cultural,مکان‌های فرهنگی (مانند تئاتر، گالری‌ها، نمایشگ...
2,2,3,Nature,مکان‌های طبیعی (مانند پارک‌ها، جنگل‌ها، دریاچه...
3,3,4,Adventure,مکان‌های ماجراجویانه (مانند کوه‌نوردی، صخره‌نو...
4,4,5,Religious,مکان‌های مذهبی (مانند کلیساها، مساجد، معابد).
5,5,6,Leisure,مکان‌های تفریحی (مانند سواحل، استراحتگاه‌ها، م...
6,6,7,Resort,استراحتگاه.


In [13]:
hotel["Tag"] = "Resort"
historic["Tag"] = "Historical"
park["Tag"] = "Nature"
museum["Tag"] = "Museum"

In [ ]:
group = pd.DataFrame(columns=["Id", "StartData", "EndDate", "ComfortRatio", "Budget"])

In [ ]:
preference = pd.DataFrame(columns=["Id", "GroupId", "UserId", "Preference"])

In [ ]:
preference.to_csv('/content/drive/MyDrive/Shiraz/preference.csv')

In [ ]:
preference

,Id,GroupId,UserId,Preference


In [ ]:
tag = pd.DataFrame(columns=["Id", "Title", "Title:Fn"])

In [ ]:
data = [
{
    "Id": 1,
    "Title": "Historical",
    "Title:Fn" : "مکان‌های تاریخی (مانند قلعه‌ها، موزه‌ها، آثار باستانی)."
},{
    "Id": 2,
    "Title": "Cultural",
    "Title:Fn" : "مکان‌های فرهنگی (مانند تئاتر، گالری‌ها، نمایشگاه‌ها)."
},{
    "Id": 3,
    "Title": "Nature",
    "Title:Fn" : "مکان‌های طبیعی (مانند پارک‌ها، جنگل‌ها، دریاچه‌ها)."
},{
    "Id": 4,
    "Title": "Adventure",
    "Title:Fn" : "مکان‌های ماجراجویانه (مانند کوه‌نوردی، صخره‌نوردی)."
},{
    "Id": 5,
    "Title": "Religious",
    "Title:Fn" : "مکان‌های مذهبی (مانند کلیساها، مساجد، معابد)."
},{
    "Id": 6,
    "Title": "Leisure",
    "Title:Fn" : "مکان‌های تفریحی (مانند سواحل، استراحتگاه‌ها، مراکز آب‌درمانی)."
}, {
    "Id": 7,
    "Title": "Resort",
    "Title:Fn" : "استراحتگاه."
}]
tag = pd.DataFrame(data)

In [ ]:
tag.to_csv('/content/drive/MyDrive/Shiraz/tag.csv')


In [ ]:
data = [{
    "Id": 1,
    "GroupId": 1,
    "UserId": 1,
    "Preference": {"Historical" : 3, "Resort": 1, "Nature": 5}
},{
    "Id": 2,
    "GroupId": 1,
    "UserId": 2,
    "Preference": {"Historical" : 2, "Cultural": 1, "Nature": 3}
},{
    "Id": 3,
    "GroupId": 1,
    "UserId": 3,
    "Preference": {"Historical" : 2, "Cultural": 5}
},{
    "Id": 4,
    "GroupId": 2,
    "UserId": 4,
    "Preference": {"Cultural": 5, "Historical": 3}
}]

In [ ]:
preference = pd.DataFrame(data)
preference.to_csv('/content/drive/MyDrive/Shiraz/preference.csv')

In [ ]:
grouped = preference.groupby('GroupId')

In [ ]:

for name, group in grouped:
  if (len(group) > 1):
    preferenceList = {}
    for preference in group['Preference']:
      for p in preference:
        if(p not in preferenceList):
          preferenceList[p] = preference[p]
        else:
          preferenceList[p] += preference[p]
    for preference in preferenceList:
      preferenceList[preference] = preferenceList[preference] / len(group)
  else:
      preferenceList = group['Preference']

preferenceList



,Preference
3,"{'Cultural': 5, 'Historical': 3}"


In [ ]:
data = [{
    "Id" : 1,
    "StartData" : "2012-02-02",
    "EndDate" : "2012-02-02",
    "ComfortRatio" : 10,
    "Budget" : 10000
}]


In [ ]:
group = pd.DataFrame(data)

In [82]:
group.to_csv('/content/drive/MyDrive/Shiraz/group.csv')

In [63]:
locations = pd.concat([hotel, historic, museum, park, resturant])
locations["Id"] = [i for i in range(len(locations))]

In [83]:
locations = locations[[
                      "Id",
                      "name",
                      "name:en",
                      "Latitude",
                      "Longitude",
                      "opening_hours",
                      "price",
                      "stars",
                      "categories",
                      "attributes",
                       "Tag"]]

In [90]:
locations.to_csv('/content/drive/MyDrive/Shiraz/locations.csv')

In [102]:
from os import replace
import random
import numpy as np
import folium

from pymoo.core.crossover import Crossover
from pymoo.core.problem import Problem
from pymoo.core.repair import Repair
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.core.population import Population
from pymoo.core.crossover import Crossover
from pymoo.operators.selection.rnd  import RandomSelection
from pymoo.core.sampling import Sampling
from geopy.distance import geodesic
from geopy.geocoders import ArcGIS
from geopy.geocoders import Nominatim
from pymoo.core.callback import Callback
from pymoo.operators.selection.tournament import TournamentSelection
from pymoo.core.selection import Selection
from pymoo.visualization.scatter import Scatter
from pymoo.core.mutation import Mutation


def repairSolution(solution):
   solution = [int(row) for row in solution]
   return solution


def isValidSolution(solution):
   solution = [int(row) for row in solution]
   return True




def createSolution(n_var):
    sample = [0 for _ in range(n_var)]


    for i in range(1, n_var-1):
      location = locations.sample(n=1)
      sample[i] = location["Id"].values[0]

    return sample


class ValidatePopulationCallback(Callback):
    def __init__(self):
        super().__init__()
        self.history = []  # نگه‌داشتن تاریخچه جمعیت‌های معتبر

    def notify(self, algorithm):
        # ترکیب والدین و فرزندان
        combined_population = np.vstack([algorithm.pop.get("X"), algorithm.off.get("X")])

        # اصلاح جمعیت جدید
        new_population = np.zeros_like(combined_population)  # جمعیت جدید اصلاح‌شده

        for i in range(len(combined_population)):
            # اصلاح هر فرد از جمعیت
            individual = combined_population[i]

            # اصلاح فرد با استفاده از روش‌های مورد نظر (مثلاً ایجاد یک راه‌حل معتبر)
            # فرض کنید تابع اصلاحی createValidSolution این کار را انجام می‌دهد
            individual = self.repair_individual(individual)

            # اصلاح فرد در جمعیت جدید
            new_population[i] = individual
        F = np.zeros((new_population.shape[0], algorithm.problem.n_obj))
        algorithm.problem._evaluate(new_population, out={"F": F})




        # به روز رسانی جمعیت با جمعیت اصلاح‌شده
        # اطمینان از هم‌خوانی اندازه جمعیت
        if new_population.shape[0] == algorithm.pop.size:
            algorithm.pop.set("X", new_population)
        else:
            # اگر تعداد تغییر کرده، تعداد جمعیت را مطابق با نیاز تنظیم کنید
            algorithm.pop.set("X", new_population[:algorithm.pop.size])


    def repair_individual(self, individual):
        if(isValidSolution(individual) == False):
            individual = repairSolution(individual)
        return individual




class CustomMutation(Mutation):
    def _do(self, problem, X, **kwargs):
      return X

class TournamentSelection(Selection):
    def __init__(self, tournament_size=2):
        super().__init__()
        self.tournament_size = tournament_size

    def _do(self, problem, pop, n_select, n_parents=1, **kwargs):
        selected = np.full((n_select, n_parents), -1, dtype=int)

        for i in range(n_select):
            for j in range(n_parents):
                # انتخاب کاندیداها به صورت تصادفی
                candidates = np.random.choice(len(pop), self.tournament_size, replace=True)

                # مقایسه کاندیداها بر اساس fitness (مقدار F)
                best = candidates[np.argmin([pop[c].F[0] for c in candidates])]
                selected[i, j] = best

        return selected




class CustomSampling(Sampling):
    def _do(self, problem, n_samples, **kwargs):
        n_var = problem.n_var  # تعداد متغیرها
        xl = problem.xl        # حد پایین متغیرها
        xu = problem.xu        # حد بالا متغیرها
        samples = np.zeros([n_samples, n_var], dtype=int)

        for row in range(len(samples)):
            samples[row] = createSolution(n_var)


        # print("Initial Population:")
        # print(samples)

        # print('---------------------')

        return samples


def getObjective(routes):
    total_distance = []
    total_cost = []
    total_rate = []


    for route in routes:


        route = route.astype(int)
        distance = 0
        cost = 0
        rate = 0


        # calculate distance
        for i in range(0, len(route)-1):
            start, end = int(route[i]), int(route[i + 1])

            start = locations[locations["Id"] == start]
            end = locations[locations["Id"] == end]

            start_lat, start_lon = start["Latitude"].values[0], start["Longitude"].values[0]
            end_lat, end_lon = end["Latitude"].values[0], end["Longitude"].values[0]

            distance += np.float64(geodesic((start_lat, start_lon), (end_lat, end_lon)).km)


        # caculate rate
        for i in range(0, len(route)):
          node = locations[locations["Id"] == route[i]]
          rate +=  node["stars"].values[0]


        # caculate cost
        for i in range(0, len(route)):
            start = locations[locations["Id"] == route[i]]
            cost += start["price"].values[0]
            print(start["price"].values[0])





        print(f"route: {route} -> d: {distance}, c: {cost}")
        total_distance.append(distance)
        total_cost.append(cost)
        total_rate.append(rate)

    return (total_distance, total_cost, total_rate)


# مدل و الگوریتم NSGA-II
class HotelRouteProblem(Problem):
    def __init__(self, n_var):
        super().__init__(n_var=n_var,
                         n_obj=3,
                         n_constr=0,
                         xl=0,
                         xu=len(locations) - 1,
                         type_var=int)



    def _evaluate(self, x, out, *args, **kwargs):
        total_distance, total_cost, total_rate = getObjective(x)
        # for i in range(len(total_distance)):
        #     print(f"total_distance: {total_distance[i]}, cost: {total_cost[i]}")
        out["F"] = np.column_stack([total_distance, total_cost, total_rate])






class CustomeCrossover(Crossover):
    def __init__(self):

        # define the crossover: number of parents and number of offsprings
        super().__init__(2,2)

    def _do(self, problem, X, **kwargs):

        _, n_matings, n_var = X.shape
        Y = np.full_like(X, None, dtype=object)

        for k in range(n_matings):
            while True:
                # انتخاب والدین
                parent1 = [int(item) for item in X[0, k]]
                parent2 = [int(item) for item in X[1, k]]



                # انتخاب نقطه تصادفی
                point = np.random.randint(1, n_var-1)

                # تولید فرزندان
                child1 = np.hstack([parent1[0], parent1[1:point], parent2[point:-1], parent1[-1]])
                child2 = np.hstack([parent2[0], parent2[1:point], parent1[point:-1], parent2[-1]])


                # print(f"parent1: {parent1}")
                # print(f"parent2: {parent2}")
                # print(f"child1:  {child1}")
                # print(f"child2:  {child2}")
                # print("--------------")
                break


            # ذخیره فرزندان
            Y[0, k] = child1
            Y[1, k] = child2

        return Y





selection  = TournamentSelection(tournament_size=1000)


algorithm = NSGA2(pop_size=20,
                  sampling=CustomSampling(),
                  crossover=CustomeCrossover(),
                  callback=ValidatePopulationCallback(),
                  mutation=CustomMutation(),
                  selection=selection)



mt = int(input("mt: "))


problem = HotelRouteProblem(n_var=mt)
res = minimize(problem,
               algorithm,
               termination=('n_gen', 3),
               verbose=True)



best_index = np.argmin(res.F[:, 2])
best_route = res.X[best_index].astype(int)
best_distance = res.F[best_index, 0]
best_cost = res.F[best_index, 1]
best_rate = res.F[best_index, 2]

print('---------------')
print("Best Route (order of locations):", best_route)
print("Best Distance:", best_distance)
print("Best Cost:", best_cost)
print("Best rate:", best_rate)

plot = Scatter()
plot.add(problem.pareto_front(), plot_type="line", color="black", alpha=0.7)
plot.add(res.F, facecolor="none", edgecolor="red")
plot.show()



mt: 5
74.0
40.0
nan
35.0
74.0
route: [  0  89 725  15   0] -> d: 4.875996427239898, c: nan
74.0
nan
35.0
nan
74.0
route: [  0 753 678 555   0] -> d: 23.50270681440768, c: nan
74.0
nan
nan
nan
74.0
route: [  0 242 512 456   0] -> d: 33.0799744777121, c: nan
74.0
nan
26.0
nan
74.0
route: [  0 563 116 535   0] -> d: 20.99591813544431, c: nan
74.0
nan
nan
nan
74.0
route: [  0 383 156 458   0] -> d: 26.17100208615405, c: nan
74.0
nan
84.0
nan
74.0
route: [  0 231  25 230   0] -> d: 7.027677783660604, c: nan
74.0
nan
nan
nan
74.0
route: [  0 375 743 532   0] -> d: 32.552662227900996, c: nan
74.0
nan
45.0
nan
74.0
route: [  0 310 132 536   0] -> d: 23.24504971580089, c: nan
74.0
nan
44.0
nan
74.0
route: [  0 404 133 508   0] -> d: 61.7680974175243, c: nan
74.0
nan
85.0
nan
74.0
route: [  0 310  63 284   0] -> d: 15.289426851522325, c: nan
74.0
nan
nan
25.0
74.0
route: [  0 194 265 637   0] -> d: 20.951561316779003, c: nan
74.0
nan
nan
nan
74.0
route: [  0 388 217 395   0] -> d: 44.93100306077

AssertionError: Invalid progress was set by the TerminationCriterion.

In [ ]:
import folium
from geopy.distance import geodesic

from geopy.geocoders import ArcGIS
from geopy.geocoders import Nominatim


geolocator = Nominatim(user_agent="location_details")

map=folium.Map(location=[h1["latitude"], h1["longitude"]], zoom_start=15)



for route in best_route:
  s = scinecSpot[scinecSpot["id"] == route]
  latitude = s["latitude"]
  longitude = s["longitude"]
  category = s["categories"]
  color = 'red'

  if(s.iloc[0]["type"] == 'r'):
    color = "blue"
  elif(s.iloc[0]["type"] == 'c'):
    color = 'black'
  map.add_child(folium.Marker(location=[latitude, longitude], popup=f"{latitude},{longitude},{category}",icon=folium.Icon(color=color)))

map.add_child(folium.Marker(location=[h1["latitude"], h1["longitude"]],icon=folium.Icon(color='green')))


In [ ]:
import folium
from geopy.distance import geodesic

from geopy.geocoders import ArcGIS
from geopy.geocoders import Nominatim


geolocator = Nominatim(user_agent="location_details")

map=folium.Map(location=[h1["latitude"], h1["longitude"]], zoom_start=15)



for index, row in scinecSpot.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    map.add_child(folium.Marker(location=[latitude, longitude], popup=f"{latitude},{longitude},{category}",icon=folium.Icon(color='red')))

map.add_child(folium.Marker(location=[h1["latitude"], h1["longitude"]],icon=folium.Icon(color='green')))




الگوریتم فازی - برای هر هتل سه مقدار بدبینانه،خوش بینانه و متوسط رو در نظر می گیریم

In [ ]:
import requests

# API Key خود را اینجا وارد کنید
API_KEY = '3k_wLMNf6CwujHWVLSLnoZFLnuDLAkzBrLxfZU9GT5cG2tkgngTk3YcAD80AUOTE665dznOmk1LXWbVHFvtBehmk-ao0Dnzo7urxabgbhvXWcDzWSRNWfvoXm-cDZ3Yx'

# URL پایه برای درخواست‌ها
url = 'https://api.yelp.com/v3/businesses/SoSQhqqOaS10aUuxDr1YtA'

# سربرگ‌ها (Headers) با کلید API
headers = {
    'Authorization': f'Bearer {API_KEY}',
}

# پارامترهای جستجو (می‌توانید موقعیت جغرافیایی، نوع کسب‌وکار و غیره را مشخص کنید)
params = {
    # 'term': 'hotels',   # نوع کسب‌وکار
    'location': 'New York',  # مکان
    'limit': 5               # تعداد نتایج
}

# ارسال درخواست به Yelp API
response = requests.get(url, headers=headers, params=params)

# بررسی وضعیت درخواست و چاپ نتایج
if response.status_code == 200:
    data = response.json()
    businesses = data.get('businesses')
    print(businesses)
    # for business in businesses:
    #     print(f"Name: {business['name']}, Rating: {business['rating']}, Address: {', '.join(business['location']['display_address'])}")
else:
    print(f"Error: {response.status_code}, {response.text}")


In [ ]:
# آرایه اصلی
# preference = {
#     'm': {'priority': 1, 'count': 0},
#     'r': {'priority': 1, 'count': 0},
#     'x': {'priority': 1, 'count': 0},
#     't': {'priority': 2, 'count': 0},
#     'h': {'priority': 5, 'count': 0},
# }


preference = {
    'm': {'priority': 2, 'count': 0},
    'r': {'priority': 1, 'count': 0}
}

# مسیر انتخاب‌شده
# route = ['h', 'm', 'h', 'r', 'h', 't', 'x']  # 'x' در ترجیحات تعریف نشده
route = ['m', 'm', 'm', 'm', 'r', 'r']
# وزن کل
totalWeight = sum(pref['priority'] for pref in preference.values())

# به‌روزرسانی تعداد تکرار
for location in route:
    if location in preference:
        preference[location]['count'] += 1

# محاسبه شباهت کل
numerator = sum(preference[p]['priority'] * preference[p]['count'] for p in preference)
denominator = totalWeight * len(route)
total_similarity = (numerator / denominator) * 100

# نمایش نتایج
for p in preference:
    similarity = (preference[p]['priority'] * preference[p]['count']) / denominator * 100
    print(f"Similarity for '{p}': {similarity:.2f}%")

print(f"\nTotal similarity: {total_similarity:.2f}%")


Similarity for 'm': 44.44%
Similarity for 'r': 11.11%

Total similarity: 55.56%
